In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics
import seaborn as sns; sns.set(style="whitegrid")
import math

%matplotlib inline

In [2]:
def value(message1=[], message2=[]):
    space = 40
    right = ''
    if (len(message1)>0):
        n = space-len(message1[1])
        right =  message1[0].ljust(n, ' ') + message1[1]
    
    lefht = ''
    if(len(message2) > 0):
        n = space-len(message2[1])
        lefht =  message2[0].ljust(n, ' ') + message2[1]
    
    print(right.ljust(n, ' ') + '   ' + lefht)


In [3]:
line = '===================================================================================='

print('Perídos: 250')
print(line)
value(['Independente', 'ABEV3'], ['Dependente', 'ITUB3'])
value(['R$', '25,00'], ['R$', '54,00'])
print(line)
value(['p-value', '-4,4485'], ['Meia Vida', '75'])
value(['ADF', '99,00%'], ['Correlação', '75%'])
value(['Aceita t0', '0,00%'], ['Inverter', 'Sim'])
print(line)
value(['Retorno', '7,64%'], ['Gain', '177,35898'])
value(['Atual', '2,41%'], ['Loss', '08,81'])
value(['Loss', '-4,20%'], ['', ''])
print(line)
value(['Ratio Entrada', '3,0170'])
value(['Ratio Saída', '3,0170'])
value(['Ratio Saída', '3,0170'])
print(line)
value(['Coef.Temp', '0,0775'], ['Coef', '0,0775'])
value(['Coef.Ang', '0,0775'], ['Intercept', '0,0775'])
value(['Coef.Lin', '0,0775'], ['', ''])
print(line)

Perídos: 250
Independente                       ABEV3   Dependente                         ITUB3
R$                                 25,00   R$                                 54,00
p-value                          -4,4485   Meia Vida                             75
ADF                               99,00%   Correlação                           75%
Aceita t0                          0,00%   Inverter                             Sim
Retorno                            7,64%   Gain                           177,35898
Atual                              2,41%   Loss                               08,81
Loss                              -4,20%                                           
Ratio Entrada                     3,0170   
Ratio Saída                       3,0170   
Ratio Saída                       3,0170   
Coef.Temp                         0,0775   Coef                              0,0775
Coef.Ang                          0,0775   Intercept                         0,0775
Coef.Lin       

In [4]:
x = [1.3,1.6,1.8,2.1,2.3,2.7]
y = [4.1,4.7,5.8,5.9,6.5,7.7]

x=[10.55]
y=[14.26]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()

In [5]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 08 Jul 2020   Prob (F-statistic):                nan
Time:                        23:11:11   Log-Likelihood:                    inf
No. Observations:                   1   AIC:                              -inf
Df Residuals:                       0   BIC:                              -inf
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3517        nan        nan        n

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:71: ValueWarning: omni_normtest is not valid with less than 8 observations; 1 samples were given.
  "samples were given." % int(n), ValueWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1648: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1657: RuntimeWarning: invalid value encountered in double_scalars
  return self.ess/self.df_model
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1665: RuntimeWarning: invalid value encountered in double_scalars
  return self.ssr/self.df_resid
C:\ProgramData\Anac

In [41]:
from statsmodels.iolib.summary import Summary

In [42]:
smry = Summary()

In [50]:
top_left = [('Dep. Variable:', None),
            ('Model:', None),
            ('Method:', ['Least Squares']),
            ('Date:', None),
            ('Time:', None),
            ('No. Observations:', None),
            ('Df Residuals:', None),
            ('Df Model:', None),
            ]

In [53]:
top_left

[('Dep. Variable:', None),
 ('Model:', None),
 ('Method:', ['Least Squares']),
 ('Date:', None),
 ('Time:', None),
 ('No. Observations:', None),
 ('Df Residuals:', None),
 ('Df Model:', None)]

In [54]:
rsquared_type = 'Meu nome'
top_right = [('R-squared' + rsquared_type + ':', ["%#8.3f" % 10]),
             ('Adj. R-squared' + rsquared_type + ':', ["%#8.3f" % 250]),
             ('F-statistic:', ["%#8.4g" % 24]),
             ('Prob (F-statistic):', ["%#6.3g" % 555]),
             ('Log-Likelihood:', None),
             ('AIC:', ["%#8.4g" % 10]),
             ('BIC:', ["%#8.4g" % 11])
             ]

In [55]:
top_right

[('R-squaredMeu nome:', ['  10.000']),
 ('Adj. R-squaredMeu nome:', [' 250.000']),
 ('F-statistic:', ['   24.00']),
 ('Prob (F-statistic):', ['  555.']),
 ('Log-Likelihood:', None),
 ('AIC:', ['   10.00']),
 ('BIC:', ['   11.00'])]

In [18]:
import TestsCodePython as test
import importlib
importlib.reload(test)

<module 'TestsCodePython' from 'D:\\trades\\pairs_trading_b3\\TestsCodePython.py'>

In [17]:
test.soma(2, 3)

10